In [1]:
from datetime import datetime
import re

import pandas as pd
import numpy as np

from modules.info import *
from modules.save2csv import *


In [2]:
entity_with_law = pd.read_csv(path + 'outputs/3_法规触发结果_' + date + '.csv')
entity_with_label = pd.read_csv(path + 'outputs/1_带标签的主体_' + date + '.csv')
model_rule = pd.read_excel(path + 'rules/模型规则_230526.xlsx', sheet_name='法规_覆盖主体_章节')

In [3]:
# 整理DataFrame
model_rule.rename(columns={'覆盖要求': 'cover_require',
                           '覆盖主体编码': 'cover_code',
                           '标签编码': 'label_code',
                           '法规编码':'law',
                           '章节编码':'chapter_code'}, inplace=True)
entity_with_label.rename(columns={'证券代码': 'stock_code',
                                  '证券简称': 'stock_name',
                                  '统一社会信用代码': 'uniform_credit'}, inplace=True)

cover_rule = model_rule[['label_code', 'cover_require', 'cover_code']]

cover_rule['cover_require'] = cover_rule['cover_require'].str.replace(r"ComFeature-[a-zA-Z]*-[a-zA-Z]*", '')
cover_rule['cover_require'] = cover_rule['cover_require'].str.replace(" = ", "")
cover_rule['cover_require'] = cover_rule['cover_require'].str.replace('"', '')
cover_rule['cover_require'] = cover_rule['cover_require'].str.replace('FALSE', '0')
cover_rule['cover_require'] = cover_rule['cover_require'].str.replace('TRUE', '1')


/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_13035/3993447800.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  cover_rule['cover_require'] = cover_rule['cover_require'].str.replace(r"ComFeature-[a-zA-Z]*-[a-zA-Z]*", '')
/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_13035/3993447800.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cover_rule['cover_require'] = cover_rule['cover_require'].str.replace(r"ComFeature-[a-zA-Z]*-[a-zA-Z]*", '')
/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_13035/3993447800.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [4]:
# 判断主体覆盖特征是否满足
entity_with_label_melted = entity_with_label.melt(id_vars=['stock_code', 'stock_name', 'uniform_credit'],
                                                  var_name='label_code',
                                                  value_name='label_value')
entity_with_label_melted['label_value'] = np.where(entity_with_label_melted['label_value'] == True, '1', entity_with_label_melted['label_value'])
entity_with_label_melted['label_value'] = np.where(entity_with_label_melted['label_value'] == False, '0', entity_with_label_melted['label_value'])

entity_with_cover = pd.merge(entity_with_label_melted, cover_rule, on=['label_code'])
entity_with_cover['whether_meet'] = np.where(entity_with_cover['label_value'] == entity_with_cover['cover_require'], True, False)

In [5]:
# 判断是否属于章节触发需覆盖的主体
grouped = entity_with_cover.groupby(['stock_code', 'stock_name', 'uniform_credit', 'cover_code'])
entity_with_cover['whether_cover'] = grouped['whether_meet'].transform('all')
entity_with_cover.drop(columns=['label_code', 'label_value', 'cover_require', 'whether_meet'], inplace=True)
entity_with_cover.drop_duplicates(inplace=True)


In [6]:
# 合并所有需要的DataFrame
entity_with_law_and_cover = pd.merge(entity_with_law, entity_with_cover, on=['stock_code', 'stock_name', 'uniform_credit'], how='left')

chapter = model_rule[['law', 'cover_code', 'chapter_code']]
chapter.drop_duplicates(inplace=True)

entity_with_law_and_cover = pd.merge(entity_with_law_and_cover, chapter, on=['law', 'cover_code'], how='left')
entity_with_law_and_cover.dropna(subset='chapter_code', inplace=True)
entity_with_law_and_cover.drop(columns=['entity_id', 'uniform_credit'], inplace=True)


/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_13035/570116477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chapter.drop_duplicates(inplace=True)


In [7]:
# 先进行章节触发判断
grouped = entity_with_law_and_cover.groupby(['stock_code', 'stock_name', 'entity_name', 'year', 'law'])
entity_with_law_and_cover['chapter_result_danger'] = grouped['law_result_danger'].transform('any')
entity_with_law_and_cover['chapter_result_normal'] = grouped['law_result_normal'].transform('any')
entity_with_law_and_cover

# 随后加入主体覆盖判断
entity_with_law_and_cover['chapter_result_danger'] = np.where(entity_with_law_and_cover['whether_cover'] == False, False, entity_with_law_and_cover['chapter_result_danger'])
entity_with_law_and_cover['chapter_result_normal'] = np.where(entity_with_law_and_cover['whether_cover'] == False, False, entity_with_law_and_cover['chapter_result_normal'])

# 删除不必要的列与重复行
entity_with_law_and_cover.drop(columns=['law', 'law_result_danger', 'law_result_normal', 'cover_code', 'whether_cover'], inplace=True)
entity_with_law_and_cover.drop_duplicates(inplace=True)

# 调整列位置
entity_with_law_and_cover.insert(0,'stock_code', entity_with_law_and_cover.pop('stock_code'))
entity_with_law_and_cover.insert(1,'stock_name', entity_with_law_and_cover.pop('stock_name'))
entity_with_law_and_cover.insert(2,'entity_name', entity_with_law_and_cover.pop('entity_name'))

entity_with_law_and_cover.head()

,stock_code,stock_name,entity_name,year,chapter_code,chapter_result_danger,chapter_result_normal
26,000002.SZ,万科A,万科企业股份有限公司,2021,BSE-IPOLaw-2,False,False
69,000002.SZ,万科A,万科企业股份有限公司,2020,BSE-IPOLaw-2,False,False
112,000002.SZ,万科A,万科企业股份有限公司,2019,BSE-IPOLaw-2,False,False
161,000006.SZ,深振业A,深圳市振业(集团)股份有限公司,2021,BSE-IPOLaw-2,False,False
204,000006.SZ,深振业A,深圳市振业(集团)股份有限公司,2020,BSE-IPOLaw-2,False,False


In [8]:
save2csv(entity_with_law_and_cover, '4_章节触发结果')